### Importing libraries

In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from apyori import apriori

from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori as ap #fpmax, fpgrowth

warnings.filterwarnings('ignore')

%matplotlib inline

### Reading input file

In [2]:
path_to_input_file = os.path.join(os.getcwd(), 'Employee_skills_traits.csv')
employee_skills_df = pd.read_csv(path_to_input_file)
employee_skills_df

,ID,Employment period,Time in current department,Gender,Team leader,Age,Member of professional organizations,.Net,SQL Server,HTML CSS Java Script,PHP mySQL,Fast working,Awards,Communicative
0,6723,5,4,0,1,48,1,1,1,0,1,1,0,1
1,8923,5,3,1,1,26,1,1,0,0,1,1,0,0
2,2322,11,8,0,0,34,0,0,1,1,0,1,0,1
3,235,7,5,1,1,27,1,0,1,0,1,1,1,0
4,9523,18,8,0,0,38,0,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,6098,12,3,1,1,32,1,0,0,0,1,1,0,1
994,12133,16,8,1,0,36,1,1,1,0,0,0,1,1
995,854,11,4,0,1,34,1,0,1,0,1,1,0,0
996,13444,8,8,1,0,36,0,0,1,0,0,0,1,1


### Removing whitespaces from column names

In [3]:
employee_skills_df.columns = employee_skills_df.columns.str.strip()
employee_skills_df.head()

,ID,Employment period,Time in current department,Gender,Team leader,Age,Member of professional organizations,.Net,SQL Server,HTML CSS Java Script,PHP mySQL,Fast working,Awards,Communicative
0,6723,5,4,0,1,48,1,1,1,0,1,1,0,1
1,8923,5,3,1,1,26,1,1,0,0,1,1,0,0
2,2322,11,8,0,0,34,0,0,1,1,0,1,0,1
3,235,7,5,1,1,27,1,0,1,0,1,1,1,0
4,9523,18,8,0,0,38,0,0,0,1,1,1,1,1


# Data Preprocessing

### Dropping duplicate records, if any

In [4]:
print("Dimension of the data before deleting duplicates - ",employee_skills_df.shape)

duplicate_rows= employee_skills_df[employee_skills_df.duplicated(['ID'],keep=False)]
print("Number of duplicate records - ", sum(employee_skills_df.duplicated(['ID'])))

if not duplicate_rows.empty:
    employee_skills_df.drop_duplicates(['ID'],keep='first',inplace=True)

print("Dimension of the data after deleting duplicates - ",employee_skills_df.shape)

Dimension of the data before deleting duplicates -  (998, 14)
Number of duplicate records -  24
Dimension of the data after deleting duplicates -  (974, 14)


### Understanding correlation between data points

In [5]:
correlation = employee_skills_df.corr()
correlation

,ID,Employment period,Time in current department,Gender,Team leader,Age,Member of professional organizations,.Net,SQL Server,HTML CSS Java Script,PHP mySQL,Fast working,Awards,Communicative
ID,1.000000,0.025959,0.047753,-0.004133,0.013425,0.058238,0.002218,0.031942,0.034973,0.017177,0.002038,0.004879,-0.018351,0.007545
Employment period,0.025959,1.000000,0.495928,0.039581,-0.047627,0.165620,-0.046843,-0.082505,-0.038466,0.052459,0.035594,0.033878,0.011274,-0.084706
Time in current department,0.047753,0.495928,1.000000,0.014418,-0.000747,0.074929,-0.066527,-0.052280,-0.043505,-0.002027,0.021906,0.050295,-0.011211,-0.019553
Gender,-0.004133,0.039581,0.014418,1.000000,0.020797,0.009926,-0.006607,-0.030317,-0.050749,0.004742,0.068526,-0.057147,0.031006,-0.025402
Team leader,0.013425,-0.047627,-0.000747,0.020797,1.000000,-0.018529,-0.040955,0.031245,-0.005795,-0.026869,0.006538,0.018381,-0.004158,0.114746
Age,0.058238,0.165620,0.074929,0.009926,-0.018529,1.000000,-0.057143,-0.001722,0.000331,-0.006531,0.007966,0.032850,0.023289,-0.003520
Member of professional organizations,0.002218,-0.046843,-0.066527,-0.006607,-0.040955,-0.057143,1.000000,-0.008934,0.015796,-0.020245,-0.066432,-0.016253,0.006249,0.006787
.Net,0.031942,-0.082505,-0.052280,-0.030317,0.031245,-0.001722,-0.008934,1.000000,0.045102,0.007182,-0.000173,0.002664,-0.024367,-0.047258
SQL Server,0.034973,-0.038466,-0.043505,-0.050749,-0.005795,0.000331,0.015796,0.045102,1.000000,0.035879,-0.049369,0.002613,0.041389,-0.068000
HTML CSS Java Script,0.017177,0.052459,-0.002027,0.004742,-0.026869,-0.006531,-0.020245,0.007182,0.035879,1.000000,-0.001127,0.005909,0.003981,0.007336


### Removing columns which are not relevant to finding associations
#### Studying the correlation table we see that ID and Gender are having negative or negligent correlations with most of the other attributes and can be considered irrelevant to finding out associations, it will be wise to drop them before applying the algorithm

In [6]:
employee_skills_df.drop(columns=['ID', 'Gender'], inplace=True)
employee_skills_df

,Employment period,Time in current department,Team leader,Age,Member of professional organizations,.Net,SQL Server,HTML CSS Java Script,PHP mySQL,Fast working,Awards,Communicative
0,5,4,1,48,1,1,1,0,1,1,0,1
1,5,3,1,26,1,1,0,0,1,1,0,0
2,11,8,0,34,0,0,1,1,0,1,0,1
3,7,5,1,27,1,0,1,0,1,1,1,0
4,18,8,0,38,0,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
993,12,3,1,32,1,0,0,0,1,1,0,1
994,16,8,0,36,1,1,1,0,0,0,1,1
995,11,4,1,34,1,0,1,0,1,1,0,0
996,8,8,0,36,0,0,1,0,0,0,1,1


# Data Conversion/Normalization
#### Here we will convert and normalize numeric attributes such as Employment Period, Age and Time in current department to categorical variables

In [7]:
# Normalizing employment period values through equal width binning

employee_skills_df['Employment period'].describe()

employment_period_bin_label = ['0-5', '6-10', '11-15', '16-20']
cut_bins_employment_period = [0, 5, 10, 15, 20]
employee_skills_df['Employment period'] = pd.cut(employee_skills_df['Employment period'], bins=cut_bins_employment_period, labels=employment_period_bin_label)
employee_skills_df.head()


,Employment period,Time in current department,Team leader,Age,Member of professional organizations,.Net,SQL Server,HTML CSS Java Script,PHP mySQL,Fast working,Awards,Communicative
0,0-5,4,1,48,1,1,1,0,1,1,0,1
1,0-5,3,1,26,1,1,0,0,1,1,0,0
2,11-15,8,0,34,0,0,1,1,0,1,0,1
3,6-10,5,1,27,1,0,1,0,1,1,1,0
4,16-20,8,0,38,0,0,0,1,1,1,1,1


In [8]:
# Normalizing age values through equal width binning

employee_skills_df['Age'].describe()

age_bin_label = ['20-30', '31-40', '41-50', '51-60']
cut_bins_age = [20, 30, 40, 50, 60]
employee_skills_df['Age'] = pd.cut(employee_skills_df['Age'], bins=cut_bins_age, labels=age_bin_label)
employee_skills_df.head()


,Employment period,Time in current department,Team leader,Age,Member of professional organizations,.Net,SQL Server,HTML CSS Java Script,PHP mySQL,Fast working,Awards,Communicative
0,0-5,4,1,41-50,1,1,1,0,1,1,0,1
1,0-5,3,1,20-30,1,1,0,0,1,1,0,0
2,11-15,8,0,31-40,0,0,1,1,0,1,0,1
3,6-10,5,1,20-30,1,0,1,0,1,1,1,0
4,16-20,8,0,31-40,0,0,0,1,1,1,1,1


In [9]:
# Normalizing employee's time in current department values through equal width binning

employee_skills_df['Time in current department'].describe()

current_department_bin_label = ['0-3', '4-6', '7-9', '10-12']
cut_bins_curr_dept = [0, 3, 6, 9, 12]
employee_skills_df['Time in current department'] = pd.cut(employee_skills_df['Time in current department'], bins=cut_bins_curr_dept, labels=current_department_bin_label)
employee_skills_df.head()


,Employment period,Time in current department,Team leader,Age,Member of professional organizations,.Net,SQL Server,HTML CSS Java Script,PHP mySQL,Fast working,Awards,Communicative
0,0-5,4-6,1,41-50,1,1,1,0,1,1,0,1
1,0-5,0-3,1,20-30,1,1,0,0,1,1,0,0
2,11-15,7-9,0,31-40,0,0,1,1,0,1,0,1
3,6-10,4-6,1,20-30,1,0,1,0,1,1,1,0
4,16-20,7-9,0,31-40,0,0,0,1,1,1,1,1


### Converting categorical variables to series of ones and zeros quantification and comparison

In [10]:
employee_skills_df = pd.get_dummies(employee_skills_df, columns=['Employment period', 'Age', 'Time in current department'])
employee_skills_df.head()

,Team leader,Member of professional organizations,.Net,SQL Server,HTML CSS Java Script,PHP mySQL,Fast working,Awards,Communicative,Employment period_0-5,...,Employment period_11-15,Employment period_16-20,Age_20-30,Age_31-40,Age_41-50,Age_51-60,Time in current department_0-3,Time in current department_4-6,Time in current department_7-9,Time in current department_10-12
0,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,1,0,0,1,0,0
1,1,1,1,0,0,1,1,0,0,1,...,0,0,1,0,0,0,1,0,0,0
2,0,0,0,1,1,0,1,0,1,0,...,1,0,0,1,0,0,0,0,1,0
3,1,1,0,1,0,1,1,1,0,0,...,0,0,1,0,0,0,0,1,0,0
4,0,0,0,0,1,1,1,1,1,0,...,0,1,0,1,0,0,0,0,1,0


In [11]:
from mlxtend.frequent_patterns import apriori as ap

support_list = [0.5,0.4,0.3,0.05]

for support in support_list:
    frequent_itemsets = ap(employee_skills_df,min_support=support,use_colnames=True)
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    print(f"Current Confidence is {support*100}%")
    print("Frequent itemsets are - ")
    print(frequent_itemsets)
    print("-----------------------------------------------------------------------------")


Current Confidence is 50.0%
Frequent itemsets are - 
    support                itemsets  length
0  0.504107           (Team leader)       1
1  0.510267  (HTML CSS Java Script)       1
2  0.506160          (Fast working)       1
3  0.503080                (Awards)       1
4  0.521561         (Communicative)       1
-----------------------------------------------------------------------------
Current Confidence is 40.0%
Frequent itemsets are - 
    support                                itemsets  length
0  0.504107                           (Team leader)       1
1  0.492813  (Member of professional organizations)       1
2  0.475359                                  (.Net)       1
3  0.477413                            (SQL Server)       1
4  0.510267                  (HTML CSS Java Script)       1
5  0.477413                             (PHP mySQL)       1
6  0.506160                          (Fast working)       1
7  0.503080                                (Awards)       1
8  0.521561 

#### We checked frequent itemsets for a couple of support values viz 50%, 40%, 30% and 5%. For support value of 0.5 , 0.4 and 0.3, there were no 2 item frequent sets, hence we will build our rules with 5% minimum support.

In [12]:
rules_mlxtend = association_rules(frequent_itemsets,  metric="lift", min_threshold=1)
rules_mlxtend.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Team leader),(.Net),0.504107,0.475359,0.247433,0.490835,1.032556,0.007801,1.030394
1,(.Net),(Team leader),0.475359,0.504107,0.247433,0.520518,1.032556,0.007801,1.034228
2,(PHP mySQL),(Team leader),0.477413,0.504107,0.242300,0.507527,1.006784,0.001633,1.006945
3,(Team leader),(PHP mySQL),0.504107,0.477413,0.242300,0.480652,1.006784,0.001633,1.006237
4,(Fast working),(Team leader),0.506160,0.504107,0.259754,0.513185,1.018008,0.004595,1.018647


In [15]:
print(f"Total rules generated are {rules_mlxtend.shape[0]}")

Total rules generated are 3252


#### Filtering rules with lift >=1 , confidence >= 0.55 and support >= 0.15

In [16]:
important_rules = rules_mlxtend[(rules_mlxtend['confidence']>=0.55) & (rules_mlxtend['lift']>=1) & 
                                (rules_mlxtend['support']>=0.15)]
print(f"Number of important rules with lift>=1, confidence>=0.55 and support>=0.15 are {important_rules.shape[0]}")

Number of important rules with lift>=1, confidence>=0.55 and support>=0.15 are 10


#### Sorting rules in descending order by confidence

In [18]:
important_rules.sort_values(by='confidence',ascending=False,inplace=True)
important_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
156,(Employment period_0-5),(Time in current department_0-3),0.262834,0.324435,0.178645,0.679688,2.094986,0.093372,2.109080
417,"(Team leader, Awards)",(Communicative),0.252567,0.521561,0.152977,0.605691,1.161305,0.021249,1.213362
418,"(Awards, Communicative)",(Team leader),0.255647,0.504107,0.152977,0.598394,1.187037,0.024104,1.234774
386,"(Fast working, Team leader)",(Communicative),0.259754,0.521561,0.154004,0.592885,1.136753,0.018527,1.175196
387,"(Fast working, Communicative)",(Team leader),0.263860,0.504107,0.154004,0.583658,1.157805,0.020990,1.191071
6,(Team leader),(Communicative),0.504107,0.521561,0.291581,0.578411,1.109001,0.028659,1.134849
142,(Employment period_0-5),(Communicative),0.262834,0.521561,0.151951,0.578125,1.108452,0.014867,1.134079
7,(Communicative),(Team leader),0.521561,0.504107,0.291581,0.559055,1.109001,0.028659,1.124615
123,(Time in current department_4-6),(Fast working),0.294661,0.506160,0.163244,0.554007,1.094529,0.014099,1.107282
157,(Time in current department_0-3),(Employment period_0-5),0.324435,0.262834,0.178645,0.550633,2.094986,0.093372,1.640455


#### Printing rules in format of {antecedents} ---> {consequents} for better business understanding

In [19]:
for ind in important_rules.index:
    print(f"{list(important_rules['antecedents'][ind])} ====> {list(important_rules['consequents'][ind])}")

['Employment period_0-5'] ====> ['Time in current department_0-3']
['Team leader', 'Awards'] ====> ['Communicative']
['Awards', 'Communicative'] ====> ['Team leader']
['Fast working', 'Team leader'] ====> ['Communicative']
['Fast working', 'Communicative'] ====> ['Team leader']
['Team leader'] ====> ['Communicative']
['Employment period_0-5'] ====> ['Communicative']
['Communicative'] ====> ['Team leader']
['Time in current department_4-6'] ====> ['Fast working']
['Time in current department_0-3'] ====> ['Employment period_0-5']
